In [1]:
from selenium import webdriver
import time
from selenium.webdriver.common.keys import Keys
from newspaper import Article
import pandas as pd
import requests
import re
from bs4 import BeautifulSoup as bs
import datetime
import csv
import sys
import schedule
import os


def Crawling() :
    
    url='https://www.newsnow.co.uk/h/Industry+Sectors/Commodities/Industrial+Metals/Aluminium?type=ln'

    options = webdriver.ChromeOptions()
    options.add_argument("headless")

    driver_path = '/usr/local/bin/chromedriver'
    driver=webdriver.Chrome(driver_path, options=options)
    driver.implicitly_wait(3)
    driver.get(url)

    res = requests.get(url)
    soup = bs(res.text, 'lxml')
    div = soup.select_one("div.rs-grid.rs-grid--skeleton.js-skeleton > main > div.rs-newsbox.js-newsbox.js-newsmain.js-central_ln_wrap > div").findAll('div', {"class" : "hl"})


    temp_link_list = []
    link_list = []
    name_list = []
    article_list = []
    date_list = []

    
    for hl in div :
        data = hl.find('a', {"class" : "hll"})
        time = hl.find('span', {'class' : "time"})
        
        date = time.text
        date = pd.to_datetime(date)
        date = date.strftime("%Y-%m-%d %H:%M:%S")
        date_list.append(date)

        temp_link_list.append(data.get('href'))
        name_list.append(data.text)


    for url in temp_link_list :
        res = requests.get(url)
        soup = bs(res.text, 'lxml')
        url = soup.select_one('script').get_text().split(" ")[13]
        urls = re.findall('(?:(?:https?|ftp):\/\/)?[\w/\-?=%.]+\.[\w/\-?=%.]+', str(url))
        urls = ''.join(urls)
        link_list.append(urls)


    for url in link_list :
        article = Article(url)
        try :
            article.download()
            article.parse()
            if len(article.text) > 30000 :
                article_list.append(article.summary)
            else :
                article_list.append(article.text)
        except :
            article_list.append('')


    df = pd.DataFrame({'link' : link_list,
                       'title': name_list,
                       'time': date_list,
                       'article': article_list
                      })
    

    return df


In [2]:
df = Crawling()

/var/folders/7n/qbcvswc55hzd4lnphhs0grb40000gn/T/ipykernel_66563/94606156.py:24: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver=webdriver.Chrome(driver_path, options=options)


In [3]:
df.sort_index(axis=0, ascending=False, ignore_index=True, inplace=True)

In [4]:
df['link'] = df['link'].astype('string')
df['title'] = df['title'].astype('string')
df['time'] = df['time'].astype('string')
df['article'] = df['article'].astype('string')

In [5]:
df

,link,title,time,article
0,https://www.spglobal.com/commodityinsights/en/...,Japanese Q3 aluminum premium to LME falls to $...,2022-07-05 13:21:00,"Platts, part of S&P Global Commodity Insights,..."
1,https://www.alcircle.com/news/lme-aluminium-be...,LME aluminium benchmark price contracts by US$...,2022-07-06 06:43:00,
2,https://www.alcircle.com/press-release/chinas-...,China’s A00 aluminium price faces a huge slump...,2022-07-06 07:13:00,
3,https://www.thehindubusinessline.com/portfolio...,Aluminium futures: Break down looks imminent,2022-07-06 07:37:00,The aluminium prices were range-bound for near...
4,https://www.alcircle.com/news/vedanta-increase...,Vedanta increases its aluminium wire rod price...,2022-07-06 08:45:00,
...,...,...,...,...
104,https://www.alcircle.com/press-release/chinas-...,China’s A00 aluminium ingot price mounts to RM...,2022-08-04 07:18:00,
105,https://www.thehindubusinessline.com/portfolio...,Aluminium futures remain range bound,2022-08-04 09:33:00,The aluminium futures (continuous contract) on...
106,https://www.alcircle.com/news/vedanta-cuts-its...,Vedanta cuts its aluminium ingot price for the...,2022-08-04 11:12:00,
107,https://www.alcircle.com/press-release/chinas-...,China’s A00 aluminium price gets boosted by RM...,2022-08-05 07:23:00,


In [6]:
df.to_csv('news_data_aluminum.csv',index=False)